In [ ]:
!pip install transformers==4.37.2 --quiet
!pip install torch --quiet
!pip install tqdm boto3 requests regex sentencepiece sacremoses --quiet
import numpy as np
import pandas as pd
from transformers import BertTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import torch.nn as nn
import tqdm, boto3, requests, regex, sentencepiece, sacremoses



In [ ]:
!pip install huggingface_hub --quiet
from huggingface_hub import PyTorchModelHubMixin

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'yiyanghkust/finbert-tone')
class BertForSequenceClassification(nn.Module, PyTorchModelHubMixin):
    def __init__(self, pretrained_model_name, num_labels=3):
        super(BertForSequenceClassification, self).__init__()
        self.num_labels = num_labels
        self.bert = torch.hub.load('huggingface/pytorch-transformers', 'model', pretrained_model_name)
        self.tokenizer = tokenizer
        self.loss_fn = nn.CrossEntropyLoss()
        self.dropout = nn.Dropout()
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 128)
        self.finaloutput = nn.Linear(128, num_labels)
        self.softmaxlayer = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, labels = None, *args, **kwargs):
        #print(kwargs)
        #print(args)
        outputs = self.bert(input_ids = input_ids, attention_mask = attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        fc1_output = nn.functional.relu(self.fc1(pooled_output))
        logits = self.finaloutput(fc1_output)
        logits = self.softmaxlayer(logits)
        loss = None
        if labels is not None:
          loss = self.loss_fn(logits.view(-1, self.num_labels), labels.view(-1))
        return {"logits":logits,
                "loss": loss
                }

def tokenize_function(examples):
     return tokenizer(examples["text"], padding = "max_length", truncation=True, max_length=128, return_tensors = "pt")

model = BertForSequenceClassification.from_pretrained("ANP1/finbert-tone-v0")

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


In [ ]:
myinputs = ["we don't expect a lot",
            "growth is strong and will continue to be",
            "the CEO had a meeting",
            "stock market meltdown"]
myinputs = tokenizer(myinputs, return_tensors = 'pt', padding = "max_length", truncation=True, max_length=128)
for row in model(**myinputs)["logits"].detach().numpy():
  print(np.argmax(row))

1
2
1
0
